In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import numpy as np
import torch
import os
import gc
import json
import pandas as pd
import time
from tqdm import tqdm
import traceback
import matplotlib.pyplot as plt
import networkx as nx
import warnings
from scipy import stats
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import bitsandbytes as bnb

In [ ]:

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

class FiedlerCorrelationAnalysis:
    """
    Analyzes correlations between Fiedler values (algebraic connectivity) and
    other attention graph properties in transformer models.
    """

    def __init__(self, model_name, output_dir="./fiedler_correlation_analysis"):
        """Initialize the analysis with model name and output directory."""
        self.model_name = model_name
        self.output_dir = output_dir
        self.model = None
        self.tokenizer = None
        self.results = {}

        os.makedirs(output_dir, exist_ok=True)

    def load_model(self, use_4bit=True):
        """Load the model and tokenizer with robust error handling."""
        print(f"Loading model: {self.model_name}")

        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, trust_remote_code=True)

            # try 4-bit quantization
            if use_4bit:
                try:
                    quantization_config = BitsAndBytesConfig(
                        load_in_4bit=True,
                        bnb_4bit_compute_dtype=torch.float16
                    )

                    self.model = AutoModelForCausalLM.from_pretrained(
                        self.model_name,
                        torch_dtype=torch.float16,
                        quantization_config=quantization_config,
                        device_map="auto",
                        trust_remote_code=True
                    )
                    print("Model loaded with 4-bit quantization")
                except (ImportError, ModuleNotFoundError) as e:
                    print(f"BitsAndBytes not available: {e}")
                    print("Falling back to standard loading...")
                    self.model = AutoModelForCausalLM.from_pretrained(
                        self.model_name,
                        torch_dtype=torch.float16,
                        device_map="auto",
                        trust_remote_code=True
                    )
            else:
                self.model = AutoModelForCausalLM.from_pretrained(
                    self.model_name,
                    torch_dtype=torch.float16,
                    device_map="auto",
                    trust_remote_code=True
                )

        except Exception as e:
            print(f"Error loading model: {e}")
            print("This might be due to an unsupported model architecture or outdated transformers library.")
            print("Try updating with: pip install --upgrade transformers")
            print("Or install from source: pip install git+https://github.com/huggingface/transformers.git")
            print("Using a dummy model for testing...")

            # Create a dummy model for testing purposes
            from transformers import AutoModelForCausalLM, AutoTokenizer

            # Try to load a simpler model that's likely to work
            fallback_model = "gpt2"
            print(f"Loading fallback model: {fallback_model}")

            try:
                self.tokenizer = AutoTokenizer.from_pretrained(fallback_model)
                self.model = AutoModelForCausalLM.from_pretrained(
                    fallback_model,
                    torch_dtype=torch.float16,
                    device_map="auto"
                )
            except Exception as e2:
                print(f"Error loading fallback model: {e2}")
                return None, None

        gc.collect()
        torch.cuda.empty_cache()

        print(f"Model loaded successfully")
        return self.model, self.tokenizer

    def compute_laplacian_eigenvalues(self, attention_matrix, threshold=0.01):
        """
        Compute Laplacian matrix eigenvalues from an attention matrix.
        """
        # Add safety checks for attention matrix
        if attention_matrix is None or attention_matrix.size == 0:
            return {
                "fiedler_value": 0.0,
                "star_likeness": 0.0,
                "degree_centralization": 0.0,
                "error": "Empty attention matrix"
            }

        # Ensure attention matrix has at least 2x2 dimensions for meaningful analysis
        if min(attention_matrix.shape) < 2:
            return {
                "fiedler_value": 0.0,
                "star_likeness": 0.0,
                "error": "Matrix too small for analysis"
            }

        try:
            # Binarize the attention matrix based on threshold to create adjacency matrix
            adjacency_matrix = (attention_matrix > threshold).astype(float)

            connection_count = np.sum(adjacency_matrix)
            connection_density = float(connection_count/adjacency_matrix.size)

            if connection_count < 2:
                return {
                    "fiedler_value": 0.0,
                    "star_likeness": 0.0,
                    "error": "Too few connections after thresholding"
                }

            # Create a graph to analyze the structure
            G = nx.DiGraph()
            rows, cols = np.where(adjacency_matrix > 0)
            edges = list(zip(rows.tolist(), cols.tolist()))
            G.add_edges_from(edges)

            # Compute degree metrics
            in_degrees = np.array([d for _, d in G.in_degree()])

            # Compute star-likeness metrics
            total_nodes = len(in_degrees)
            max_in_degree = np.max(in_degrees) if len(in_degrees) > 0 else 0

            # Basic star-likeness
            star_likeness = max_in_degree / max(1, total_nodes - 1)

            # Calculate degree centralization
            max_possible_diff = (total_nodes - 1) * (total_nodes - 1)
            if max_possible_diff > 0 and max_in_degree > 0:
                sum_diff = sum(max_in_degree - d for d in in_degrees)
                degree_centralization = sum_diff / max_possible_diff
            else:
                degree_centralization = 0.0

            # Calculate degree variance
            degree_variance = np.var(in_degrees) if len(in_degrees) > 1 else 0.0

            # Symmetrized adjacency for a sensible Laplacian
            adj_sym = (adjacency_matrix + adjacency_matrix.T) / 2.0

            # Compute the Laplacian
            degree_matrix = np.diag(np.sum(adj_sym, axis=1))
            laplacian_matrix = degree_matrix - adj_sym

            # Compute eigenvalues of the Laplacian
            eigenvalues = np.linalg.eigvalsh(laplacian_matrix)
            eigenvalues = np.sort(eigenvalues)  # Sort in ascending order

            # Second eigenvalue (algebraic connectivity)
            fiedler_value = float(eigenvalues[1]) if len(eigenvalues) > 1 else 0.0

            return {
                "fiedler_value": fiedler_value,
                "star_likeness": float(star_likeness),
                "degree_centralization": float(degree_centralization),
                "degree_variance": float(degree_variance),
                "connection_density": connection_density
            }

        except Exception as e:
            return {
                "fiedler_value": 0.0,
                "star_likeness": 0.0,
                "error": str(e)
            }

    def analyze_text(self, text, thresholds=[0.01, 0.05]):
        """
        Analyze a text sample and compute metrics for each layer's attention matrix.
        """
        if self.model is None or self.tokenizer is None:
            print("Model or tokenizer not loaded!")
            return {"error": "Model or tokenizer not loaded"}

        try:
            inputs = self.tokenizer(text, return_tensors="pt").to(self.model.device)

            with torch.no_grad():
                outputs = self.model(
                    **inputs,
                    output_attentions=True,
                    return_dict=True
                )

            # Extract attention patterns
            attentions = outputs.attentions  # tuple of (layer, batch, head, seq_len, seq_len)

            # Initialize results structure
            results = {
                "thresholds": thresholds,
                "layers": {}
            }

            # Process each layer
            for layer_idx, layer_attention in enumerate(attentions):
                # Each layer has shape (batch, head, seq_len, seq_len)
                layer_attention = layer_attention[0].cpu().numpy()  # shape: (head, seq_len, seq_len)

                # Compute average attention pattern across all heads
                avg_attention = np.mean(layer_attention, axis=0)

                # Store layer results
                results["layers"][str(layer_idx)] = {}

                # Analyze for each threshold
                for threshold in thresholds:
                    # Compute Laplacian eigenvalues and graph metrics
                    metrics = self.compute_laplacian_eigenvalues(
                        avg_attention,
                        threshold=threshold
                    )

                    results["layers"][str(layer_idx)][str(threshold)] = metrics

            return results

        except Exception as e:
            print(f"Error analyzing text: {e}")
            return {"error": str(e)}

    def analyze_samples(self, texts, thresholds=[0.01, 0.05]):
        """
        Analyze multiple text samples and collect metrics for correlation analysis.
        """
        print(f"Analyzing {len(texts)} text samples...")

        if self.model is None:
            self.load_model()

        # Initialize data collectors for each threshold
        metric_collectors = {}
        for threshold in thresholds:
            threshold_key = str(threshold)
            metric_collectors[threshold_key] = {}

        # Analyze each text
        successful_analyses = 0
        for i, text in enumerate(tqdm(texts, desc="Processing texts")):
            try:
                # Process text
                result = self.analyze_text(text, thresholds)

                if "error" in result:
                    continue

                successful_analyses += 1

                for layer_idx, layer_data in result["layers"].items():
                    if layer_idx not in metric_collectors[str(thresholds[0])]:
                        for threshold in thresholds:
                            threshold_key = str(threshold)
                            metric_collectors[threshold_key][layer_idx] = {
                                "fiedler_values": [],
                                "star_likeness": [],
                                "degree_centralization": [],
                                "degree_variance": [],
                                "connection_densities": []
                            }

                    for threshold in thresholds:
                        threshold_key = str(threshold)
                        if threshold_key in layer_data:
                            metrics = layer_data[threshold_key]

                            collectors = metric_collectors[threshold_key][layer_idx]

                            if "fiedler_value" in metrics:
                                collectors["fiedler_values"].append(metrics["fiedler_value"])

                            if "star_likeness" in metrics:
                                collectors["star_likeness"].append(metrics["star_likeness"])

                            if "degree_centralization" in metrics:
                                collectors["degree_centralization"].append(metrics["degree_centralization"])

                            if "degree_variance" in metrics:
                                collectors["degree_variance"].append(metrics["degree_variance"])

                            if "connection_density" in metrics:
                                collectors["connection_densities"].append(metrics["connection_density"])

                torch.cuda.empty_cache()
                gc.collect()

                if (i+1) % 5 == 0:
                    print(f"Processed {i+1}/{len(texts)} samples")

            except Exception as e:
                print(f"Error analyzing text {i+1}: {str(e)}")

        print(f"Successfully analyzed {successful_analyses}/{len(texts)} samples")

        if successful_analyses > 2:
            correlation_results = self.compute_correlations(metric_collectors, thresholds)

            self.results = {
                "metric_collectors": metric_collectors,
                "correlation_results": correlation_results,
                "thresholds": thresholds
            }

            return correlation_results
        else:
            print("Not enough successful analyses to compute correlations")
            return {}

    def compute_correlations(self, metric_collectors, thresholds):
        """
        Compute correlations between Fiedler values and other graph metrics,
        including p-values for statistical significance testing.
        """
        print("Computing correlations...")

        correlation_results = {}

        for threshold in thresholds:
            threshold_key = str(threshold)
            correlation_results[threshold_key] = {
                "by_layer": {},
                "overall": {
                    "fiedler_vs_star": {"corr": None, "p_value": None},
                    "fiedler_vs_centralization": {"corr": None, "p_value": None},
                    "fiedler_vs_variance": {"corr": None, "p_value": None},
                    "fiedler_vs_density": {"corr": None, "p_value": None}
                }
            }

            # Process each layer
            for layer_idx, layer_data in metric_collectors[threshold_key].items():
                correlation_results[threshold_key]["by_layer"][layer_idx] = {}

                # Calculate correlations for this layer if we have enough data
                fiedler_values = layer_data["fiedler_values"]

                # We need at least 3 samples for meaningful correlation
                if len(fiedler_values) >= 3:
                    correlations = {}

                    # Correlation with star-likeness
                    if len(layer_data["star_likeness"]) >= 3:
                        if len(fiedler_values) != len(layer_data["star_likeness"]):
                            min_length = min(len(fiedler_values), len(layer_data["star_likeness"]))
                            f_values = fiedler_values[:min_length]
                            s_values = layer_data["star_likeness"][:min_length]
                        else:
                            f_values = fiedler_values
                            s_values = layer_data["star_likeness"]

                        if np.std(f_values) > 0 and np.std(s_values) > 0:
                            # Use scipy.stats.pearsonr instead of np.corrcoef to get p-value
                            corr, p_value = stats.pearsonr(f_values, s_values)
                            correlations["fiedler_vs_star"] = {
                                "corr": float(corr),
                                "p_value": float(p_value)
                            }

                    # Correlation with degree centralization
                    if len(layer_data["degree_centralization"]) >= 3:
                        if len(fiedler_values) != len(layer_data["degree_centralization"]):
                            min_length = min(len(fiedler_values), len(layer_data["degree_centralization"]))
                            f_values = fiedler_values[:min_length]
                            c_values = layer_data["degree_centralization"][:min_length]
                        else:
                            f_values = fiedler_values
                            c_values = layer_data["degree_centralization"]

                        if np.std(f_values) > 0 and np.std(c_values) > 0:
                            corr, p_value = stats.pearsonr(f_values, c_values)
                            correlations["fiedler_vs_centralization"] = {
                                "corr": float(corr),
                                "p_value": float(p_value)
                            }

                    # Correlation with degree variance
                    if len(layer_data["degree_variance"]) >= 3:
                        if len(fiedler_values) != len(layer_data["degree_variance"]):
                            min_length = min(len(fiedler_values), len(layer_data["degree_variance"]))
                            f_values = fiedler_values[:min_length]
                            v_values = layer_data["degree_variance"][:min_length]
                        else:
                            f_values = fiedler_values
                            v_values = layer_data["degree_variance"]

                        if np.std(f_values) > 0 and np.std(v_values) > 0:
                            corr, p_value = stats.pearsonr(f_values, v_values)
                            correlations["fiedler_vs_variance"] = {
                                "corr": float(corr),
                                "p_value": float(p_value)
                            }

                    # Correlation with connection density
                    if len(layer_data["connection_densities"]) >= 3:
                        if len(fiedler_values) != len(layer_data["connection_densities"]):
                            min_length = min(len(fiedler_values), len(layer_data["connection_densities"]))
                            f_values = fiedler_values[:min_length]
                            d_values = layer_data["connection_densities"][:min_length]
                        else:
                            f_values = fiedler_values
                            d_values = layer_data["connection_densities"]

                        if np.std(f_values) > 0 and np.std(d_values) > 0:
                            corr, p_value = stats.pearsonr(f_values, d_values)
                            correlations["fiedler_vs_density"] = {
                                "corr": float(corr),
                                "p_value": float(p_value)
                            }

                    # Store layer-specific correlations
                    correlation_results[threshold_key]["by_layer"][layer_idx] = correlations

            # Calculate overall correlations for this threshold
            overall = correlation_results[threshold_key]["overall"]

            star_corrs = []
            star_p_values = []
            central_corrs = []
            central_p_values = []
            var_corrs = []
            var_p_values = []
            density_corrs = []
            density_p_values = []

            for layer_corrs in correlation_results[threshold_key]["by_layer"].values():
                if "fiedler_vs_star" in layer_corrs:
                    star_corrs.append(layer_corrs["fiedler_vs_star"]["corr"])
                    star_p_values.append(layer_corrs["fiedler_vs_star"]["p_value"])
                if "fiedler_vs_centralization" in layer_corrs:
                    central_corrs.append(layer_corrs["fiedler_vs_centralization"]["corr"])
                    central_p_values.append(layer_corrs["fiedler_vs_centralization"]["p_value"])
                if "fiedler_vs_variance" in layer_corrs:
                    var_corrs.append(layer_corrs["fiedler_vs_variance"]["corr"])
                    var_p_values.append(layer_corrs["fiedler_vs_variance"]["p_value"])
                if "fiedler_vs_density" in layer_corrs:
                    density_corrs.append(layer_corrs["fiedler_vs_density"]["corr"])
                    density_p_values.append(layer_corrs["fiedler_vs_density"]["p_value"])

            if len(star_corrs) > 0:
                overall["fiedler_vs_star"] = {
                    "corr": float(np.mean(star_corrs)),
                    "p_value": float(np.mean(star_p_values))
                }
            if len(central_corrs) > 0:
                overall["fiedler_vs_centralization"] = {
                    "corr": float(np.mean(central_corrs)),
                    "p_value": float(np.mean(central_p_values))
                }
            if len(var_corrs) > 0:
                overall["fiedler_vs_variance"] = {
                    "corr": float(np.mean(var_corrs)),
                    "p_value": float(np.mean(var_p_values))
                }
            if len(density_corrs) > 0:
                overall["fiedler_vs_density"] = {
                    "corr": float(np.mean(density_corrs)),
                    "p_value": float(np.mean(density_p_values))
                }

        return correlation_results

    def generate_correlation_report(self):
        """
        Generate a simple text report with correlation values and p-values.
        """
        if not self.results or "correlation_results" not in self.results:
            return "No correlation results available. Run analyze_samples first."

        correlation_results = self.results["correlation_results"]
        thresholds = self.results["thresholds"]

        lines = []
        lines.append(f"FIEDLER VALUE CORRELATION ANALYSIS FOR {self.model_name}\n")

        for threshold in thresholds:
            threshold_key = str(threshold)
            if threshold_key not in correlation_results:
                continue

            lines.append(f"THRESHOLD: {threshold}")
            lines.append("=" * 20)

            # Overall correlations
            overall = correlation_results[threshold_key]["overall"]
            lines.append("\nOVERALL CORRELATIONS:")

            for corr_name, corr_data in overall.items():
                if corr_data is not None and isinstance(corr_data, dict) and "corr" in corr_data:
                    corr_value = corr_data["corr"]
                    p_value = corr_data.get("p_value", None)

                    if p_value is not None:
                        significant = "**" if p_value < 0.05 else ""
                        lines.append(f"{corr_name}: {corr_value:.4f} (p={p_value:.4f}){significant}")
                    else:
                        lines.append(f"{corr_name}: {corr_value:.4f}")
                elif corr_data is not None:  # Handle legacy format where corr_data is just a float
                    lines.append(f"{corr_name}: {corr_data:.4f}")

            lines.append("\nLAYER-SPECIFIC CORRELATIONS (sample):")
            lines.append("(** indicates p < 0.05, statistically significant)")

            # Just show first few layers as samples
            layer_count = 0
            for layer_idx, layer_corrs in correlation_results[threshold_key]["by_layer"].items():
                if not layer_corrs or layer_count >= 5:  
                    continue

                lines.append(f"\nLayer {layer_idx}:")
                for corr_name, corr_data in layer_corrs.items():
                    if isinstance(corr_data, dict) and "corr" in corr_data:
                        corr_value = corr_data["corr"]
                        p_value = corr_data.get("p_value", None)

                        if p_value is not None:
                            significant = "**" if p_value < 0.05 else ""
                            lines.append(f"  {corr_name}: {corr_value:.4f} (p={p_value:.4f}){significant}")
                        else:
                            lines.append(f"  {corr_name}: {corr_value:.4f}")
                    else:  # Handle legacy format
                        lines.append(f"  {corr_name}: {corr_data:.4f}")

                layer_count += 1

            if layer_count == 3:
                lines.append("\n(Additional layers omitted for brevity)")

            lines.append("\n")

        report_path = os.path.join(self.output_dir, "fiedler_correlations.txt")
        with open(report_path, 'w') as f:
            f.write('\n'.join(lines))

        print(f"Report saved to {report_path}")
        return '\n'.join(lines)

    def run_analysis(self, texts=None, thresholds=[0.01, 0.05]):
        """
        Run the complete correlation analysis pipeline.
        """
        start_time = time.time()

        if self.model is None:
            self.load_model()

            if self.model is None:
                return "Model loading failed. Cannot continue analysis."

        # Use example texts if none provided
        if texts is None or not texts:
            texts = [
                "The concept of eigenvalues relates to how transformations affect spaces.",
                "Algebraic connectivity measures how well-connected a graph is.",
                "Star graphs have one central node connected to all other nodes.",
                "In transformers, attention mechanisms create dynamic connections between tokens."
            ]

        correlation_results = self.analyze_samples(texts, thresholds)

        if correlation_results:
            report = self.generate_correlation_report()
        else:
            report = "Analysis did not produce valid correlation results."

        total_time = time.time() - start_time
        print(f"Analysis completed in {total_time/60:.2f} minutes")

        return report

    def cleanup(self):
        """Clean up resources safely."""
        if self.model is not None:
            try:
                has_meta_params = False
                for param in self.model.parameters():
                    if hasattr(param, 'device') and param.device == torch.device("meta"):
                        has_meta_params = True
                        break

                if not has_meta_params:
                    self.model = self.model.to("cpu")
            except Exception as e:
                print(f"Warning: Could not move model to CPU: {e}")
                print("This is normal for models with parameters offloaded to disk")

            del self.model
            self.model = None

        if self.tokenizer is not None:
            del self.tokenizer
            self.tokenizer = None

        gc.collect()
        torch.cuda.empty_cache()

        print("Resources cleaned up")


def get_sample_texts_from_dataset(dataset_path, n_samples=10):
    """
    Extract sample texts from a dataset for analysis.
    """
    try:
        data = pd.read_csv(dataset_path)
        print(f"Loaded dataset with {len(data)} rows")

        if 'text' not in data.columns:
            print("Error: No 'text' column found in dataset")
            return []

        if len(data) > n_samples:
            samples = data.sample(n_samples)
        else:
            samples = data

        texts = samples['text'].tolist()
        return texts

    except Exception as e:
        print(f"Error loading dataset: {e}")
        return []



In [ ]:
def main():
    """Run the Fiedler value correlation analysis."""
    try:
        from google.colab import drive
        # Mount Google Drive if in Colab
        drive.mount('/content/drive')
        is_colab = True
    except ImportError:
        is_colab = False
        print("Not running in Google Colab, skipping drive mount")

    model_name = "EleutherAI/pythia-12b"  
    print(f"Using model: {model_name}")

    if is_colab:
        dataset_path = "/content/drive/MyDrive/wiki_dataset_position.csv"
        output_dir = f"/content/drive/MyDrive/Sink/fiedler/{model_name.split('/')[-1]}"
    else:
        dataset_path = "./dataset.csv"
        output_dir = f"./fiedler_corr_{model_name.split('/')[-1]}"

    analyzer = FiedlerCorrelationAnalysis(
        model_name=model_name,
        output_dir=output_dir
    )

    # Get sample texts from dataset
    n_samples = 500  
    texts = get_sample_texts_from_dataset(dataset_path, n_samples=n_samples)

    # If no texts could be loaded, use some default examples
    if not texts:
        texts = [
            "The concept of eigenvalues relates to how transformations affect spaces.",
            "Algebraic connectivity measures how well-connected a graph is.",
            "Star graphs have one central node connected to all other nodes.",
            "In transformers, attention mechanisms create dynamic connections between tokens."
        ]

    # Run analysis with specified thresholds
    try:
        thresholds = [0.001, 0.005, 0.01, 0.02, 0.5, 0.1, 0.2]
        print(f"Running analysis with thresholds: {thresholds}")

        correlation_report = analyzer.run_analysis(texts=texts, thresholds=thresholds)

        print("\nSIMPLE CORRELATION SUMMARY:")
        print("==========================")

        print(correlation_report)

        print("\nAnalysis completed successfully!")
    except Exception as e:
        print(f"Analysis error: {e}")
        traceback.print_exc()
    finally:
        analyzer.cleanup()


if __name__ == "__main__":
    main()